# Is rounding broken?

In the lecture we had the case:

In [ ]:

round(1.45, 1)                                                                                              

1.4

Why is the result 1.4 and not 1.5?

Indeed, 1.5 is what you should expect here.

Just to prove that `round()` is actually rounding to the nearest number of specified precision:

In [ ]:
round(1.4500000000001, 1)

1.5

So `round()` is indeed what we think it is, and that's different from `ceil()` and `floor()` - functions which round to the next largest value or the next smallest value:

In [ ]:
import math
math.ceil(1.5)

2

In [ ]:
math.floor(1.5)

1

Also `round()` seems to work as expected for other values:

In [ ]:

round(1.55, 1)

1.6

So what is wrong with `round(1.45, 1)`?

You may know that numbers such as $\pi = 3.141592\ldots$ cannot be represented with a finite decimal notation, but just up to a certain precision. Similarely $\frac{1}{3} = 0.3333333\ldots$ can't be represented with a finite decimal notation (although there is a precice notation in the form of $\frac{1}{3}$, but this is not a decimal notation). In other words, there are numbers which are not a [decimal fraction](https://en.wikipedia.org/wiki/Decimal#Decimal_fractions).

Computers use (non-integer) numbers in most cases in a way which is very similar to a finite decimal representation. You can imagine the computer being able to calculate decimal numbers with a certain length, i.e. a certain number of decimal digits per number. For example, if the computer can do computations with numbers up to 5 decimal digits, the computer would use `0.3333` to represent $\frac{1}{3}$, because it is the closest decimal number with 5 digits. Of course, this would introduce a numerical error, which is hopefully (!) small. This could be called _rounding error_, but it would be a bit sloppy and misleading, because there is no explicit rounding taking place - the error is caused only by the computational precision introduced by the fact that the computer can handle only numbers with a finite decimal representation. Usually we refer to this error as [numerical error](https://en.wikipedia.org/wiki/Numerical_error).

So for $\frac{1}{3} = 0.3333\ldots$ this is error what we expact, but $1.45$ obviously seems to have a finite decimal representation! After all only three digits are needed to display this number! What's the problem with $1.45$ then?

There is a subtle but important different: the computer doesn't use a finite decimal representation, but a finite binary representation. It works pretty much the same as the decimal representation with digits from 0-9 but using only 0s and 1s. Numbers which have a finite decimal representation, do not have necessarily a finite binary representation and vice versa. With $1.45$ we were (un)lucky to use a number, which has a finite decimal represantation, but the closes number which has a finite binary representation, is actually smaller than 1.45:

In [ ]:
from decimal import Decimal

Decimal(1.45)

Decimal('1.4499999999999999555910790149937383830547332763671875')


So this value gives a better imagination for the number, the computer is actually using behind the scenes even though it's internally represented as binary fraction.

Also note that:

In [ ]:
1 + 2 == 3

True

...but:

In [ ]:
0.1 + 0.2 == 0.3

False

So to sum up: When writing 1.45, the closest binary number will be used. In this case it is less then 1.45, so the value will be rounded down.

Now after writing all of this, I noticed that the official Python documentation has [a better explanation](https://docs.python.org/3/tutorial/floatingpoint.html#representation-error) of this issue... :)

**Note:** The comment about decimal numbers and binary numbers applies for most computers and most programming languages, in particular for Python. Computers have a second way to represent numbers without comma, so called _integers_. Precision accuracy is not an issue for integers - at least not in Python. There are also some programming languages and Python modules where things are slightly different.

## Part 2: round() rounds towards the even choice

The above description is the reason why 1.45 is rounded to 1.4 and not 1.5. But confusingly there is a different reason why:

In [ ]:
round(4.5)

4

The [Python documentation](https://docs.python.org/3/library/functions.html#round) for `round()` says:

> For the built-in types supporting round(number, ndigitis), values are rounded to the closest multiple of 10 to the power minus ndigits; if two multiples are equally close, rounding is done toward the even choice (so, for example, both round(0.5) and round(-0.5) are 0, and round(1.5) is 2).

So whenever number is exactly between two choices, it will not round away form zero (the choice with the larger absolute value), but take the choice with the even digit at the end.

In [ ]:
print(round(1.5), round(2.5), round(3.5), round(4.5), round(5.5), round(6.5), round(7.5), round(8.5))

2 2 4 4 6 6 8 8


Why is [rounding half to even](https://en.wikipedia.org/wiki/Rounding#Round_half_to_even) even a good idea? It avoids bias!

We haven't discussed the code below in the lecture yet, but bascially it caclulates the average of numbers from 0 to 10 and then adds 0.5 and rounds it. The average increases just by 0.5 as it should, not by 1. as it would if you would round away from 0.

In [ ]:
import numpy as np

np.mean([i for i in range(10)])

4.5

In [ ]:

np.mean([round(i + 0.5) for i in range(10)])

5.0

So here you would expect 1.0 as result, right?

In [ ]:
round(1.05, 1)

1.1

No! Because of the numeric inaccuracy again :)

Using a `Decimal()` object instead of a normal float number can do it correctly:

In [ ]:
round(Decimal('1.05'), 1)

Decimal('1.0')

**Note:** The behavior of `round()` changed between Python 2 and Python 3. In Python 2 `round()` rounds away from zero. But Python 2 is no longer used since [more than a year](https://pythonclock.org/).